**대규모 텐서플로 모델 훈련과 배포**

# 설정

In [1]:
from tensorflow.config import list_physical_devices

list_physical_devices('GPU')

2025-08-26 03:29:41.929061: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756178982.070006      35 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756178982.110370      35 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756178982.425779      35 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756178982.425806      35 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756178982.425808      35 computation_placer.cc:177] computation placer alr

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# 텐서플로 모델 서빙하기

## 텐서플로 서빙 사용

가장 먼저 모델을 빌드하고 학습한 다음 SavedModel 포맷으로 내보낸다.

### SavedModel 내보내기

MNIST 데이터 세트를 로드하고, 스케일을 조정하고, 분할한다.

In [2]:
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.backend import clear_session
from tensorflow.keras import Sequential, Input
from tensorflow import uint8
from tensorflow.keras.layers import Flatten, Rescaling, Dense
from pathlib import Path

mnist = load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
clear_session()
model = Sequential([
    Input((28, 28), dtype=uint8),
    Flatten(),
    Rescaling(1 / 255),
    Dense(100, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))  # epochs=10
model_name = 'my_mnist_model'
model_version = '0001'
model_path = Path(model_name) / model_version
model.export(model_path)

I0000 00:00:1756178986.220986      35 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5555 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti, pci bus id: 0000:06:00.0, compute capability: 8.6
I0000 00:00:1756178987.640871     139 service.cc:152] XLA service 0x7175d4005590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756178987.640916     139 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2025-08-26 03:29:47.677816: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1756178987.742497     139 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-08-26 03:29:48.410858: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38_0

  80/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.2023 - loss: 2.1788

I0000 00:00:1756178990.756577     139 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1706/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7186 - loss: 1.0548

2025-08-26 03:29:54.612010: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 72 bytes spill stores, 72 bytes spill loads

2025-08-26 03:29:54.697321: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38_0', 92 bytes spill stores, 92 bytes spill loads

2025-08-26 03:29:54.737900: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 8 bytes spill stores, 8 bytes spill loads

2025-08-26 03:29:54.762234: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_38', 396 bytes spill stores, 396 bytes spill loads

2025-08-26 03:29:54.946196: I external/local_x

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7195 - loss: 1.0519

2025-08-26 03:29:58.715729: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-08-26 03:29:58.749033: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 296 bytes spill stores, 296 bytes spill loads

2025-08-26 03:29:58.817648: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 8 bytes spill stores, 8 bytes spill loads

2025-08-26 03:29:58.953453: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_34', 792 bytes spill stores, 844 bytes spill loads

2025-08-26 03:29:59.149940: I external/local_xla

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8296 - loss: 0.6695 - val_accuracy: 0.9022 - val_loss: 0.3663
INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


INFO:tensorflow:Assets written to: my_mnist_model/0001/assets


Saved artifact at 'my_mnist_model/0001'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  124754709362064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754709361872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754709357840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754709362832: TensorSpec(shape=(), dtype=tf.resource, name=None)


파일 트리를 살핀다:

In [3]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index']

SavedModel을 검사한다:

In [4]:
!saved_model_cli show --dir '{model_path}'

2025-08-26 03:30:00.143754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756179000.169031     444 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756179000.173691     444 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756179000.185992     444 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756179000.186040     444 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756179000.186047     444 computation_placer.cc:177] computation placer alr

In [5]:
!saved_model_cli show --dir '{model_path}' --tag_set serve

2025-08-26 03:30:03.227763: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756179003.242215     461 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756179003.246319     461 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756179003.258156     461 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756179003.258205     461 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756179003.258211     461 computation_placer.cc:177] computation placer alr

In [6]:
!saved_model_cli show --dir '{model_path}' --tag_set serve --signature_def serving_default

2025-08-26 03:30:05.886667: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756179005.900237     477 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756179005.904271     477 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756179005.915854     477 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756179005.915898     477 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1756179005.915904     477 computation_placer.cc:177] computation placer alr

더 자세한 내용을 보려면 다음 명령을 실행한다:
```ipython
!saved_model_cli show --dir '{model_path}' --all
```

### 텐서플로 서빙 설치 및 시작하기

도커를 사용해 TF 서빙을 설치하려면 먼저 [Docker](https://docs.docker.com/install/)가 설치되어 있는지 확인한 후 터미널에서 다음 명령을 실행한다.
```bash
docker pull tensorflow/serving:latest-gpu

docker run -e MODEL_NAME=my_mnist_model --gpus all -it -p 8500:8500 -p 8501:8501 -v path\to\my_mnist_model:/models/my_mnist_model tensorflow/serving:latest-gpu
```

### REST API로 TF 서빙에 쿼리하기

다음으로 TF 서빙에 REST 쿼리를 전송한다:

In [7]:
from json import dumps

X_new = X_test[:3]  # 분류할 새로운 숫자 이미지가 3개 있다고 가정한다.
request_json = dumps({'signature_name': 'serving_default', 'instances': X_new.tolist()})

In [8]:
request_json[:100] + '...' + request_json[-10:]

'{"signature_name": "serving_default", "instances": [[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..., 0, 0]]]}'

이제 텐서플로 서빙의 REST API를 사용하여 예측한다:

In [9]:
from requests import post

server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = post(server_url, request_json)
response.raise_for_status()  # 오류 발생 시 예외 발생
response = response.json()

In [10]:
from numpy import array

y_proba = array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.01],
       [0.04, 0.  , 0.8 , 0.02, 0.  , 0.06, 0.06, 0.  , 0.02, 0.  ],
       [0.  , 0.93, 0.02, 0.01, 0.  , 0.  , 0.  , 0.01, 0.01, 0.  ]])

### gRPC API로 TF 서빙에 쿼리하기

In [11]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest
from tensorflow import make_tensor_proto

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = 'serving_default'
input_name = 'keras_tensor'
request.inputs[input_name].CopyFrom(make_tensor_proto(X_new))

In [12]:
from grpc import insecure_channel
from tensorflow_serving.apis.prediction_service_pb2_grpc import PredictionServiceStub

channel = insecure_channel('172.17.0.3:8500')
predict_service = PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout=10.0)

응답을 텐서로 변환한다:

In [13]:
from tensorflow import make_ndarray

output_name = 'output_0'
outputs_proto = response.outputs[output_name]
y_proba = make_ndarray(outputs_proto)

In [14]:
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.01],
       [0.04, 0.  , 0.8 , 0.02, 0.  , 0.06, 0.06, 0.  , 0.02, 0.  ],
       [0.  , 0.93, 0.02, 0.01, 0.  , 0.  , 0.  , 0.01, 0.01, 0.  ]],
      dtype=float32)

### 새 모델 버전 배포

In [15]:
# 새로운 MNIST 모델 버전 빌드 및 훈련
model = Sequential([
    Input((28, 28), dtype=uint8),
    Flatten(),
    Rescaling(1 / 255),
    Dense(50, activation='relu'),
    Dense(50, activation='relu'),
    Dense(10, activation='softmax')
])
model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))  # epochs=10

2025-08-26 03:30:11.733696: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_40', 944 bytes spill stores, 944 bytes spill loads

2025-08-26 03:30:11.820326: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_40', 488 bytes spill stores, 488 bytes spill loads

2025-08-26 03:30:11.915805: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_40', 940 bytes spill stores, 940 bytes spill loads

2025-08-26 03:30:11.931325: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_40', 272 bytes spill stores, 272 bytes spill loads

2025-08-26 03:30:12.081328: I external/l

1701/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6600 - loss: 1.2093

2025-08-26 03:30:18.080638: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_40', 272 bytes spill stores, 272 bytes spill loads

2025-08-26 03:30:18.468659: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_40', 988 bytes spill stores, 988 bytes spill loads

2025-08-26 03:30:18.478523: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_40', 484 bytes spill stores, 484 bytes spill loads

2025-08-26 03:30:18.567163: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_40', 944 bytes spill stores, 944 bytes spill loads

2025-08-26 03:30:18.696595: I external/l

1719/1719 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6616 - loss: 1.2042

2025-08-26 03:30:22.173095: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 348 bytes spill stores, 348 bytes spill loads

2025-08-26 03:30:22.235963: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads

2025-08-26 03:30:22.423571: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 816 bytes spill stores, 816 bytes spill loads

2025-08-26 03:30:22.462491: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_36', 272 bytes spill stores, 272 bytes spill loads



1719/1719 ━━━━━━━━━━━━━━━━━━━━ 12s 5ms/step - accuracy: 0.8082 - loss: 0.7184 - val_accuracy: 0.9030 - val_loss: 0.3458


In [16]:
model_version = '0002'
model_path = Path(model_name) / model_version
model.export(model_path)

INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


INFO:tensorflow:Assets written to: my_mnist_model/0002/assets


Saved artifact at 'my_mnist_model/0002'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  124754704634832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754704632336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754709358800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754704633104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754704632912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754709359376: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [17]:
sorted([str(path) for path in model_path.parent.glob('**/*')])

['my_mnist_model/0001',
 'my_mnist_model/0001/assets',
 'my_mnist_model/0001/fingerprint.pb',
 'my_mnist_model/0001/saved_model.pb',
 'my_mnist_model/0001/variables',
 'my_mnist_model/0001/variables/variables.data-00000-of-00001',
 'my_mnist_model/0001/variables/variables.index',
 'my_mnist_model/0002',
 'my_mnist_model/0002/assets',
 'my_mnist_model/0002/fingerprint.pb',
 'my_mnist_model/0002/saved_model.pb',
 'my_mnist_model/0002/variables',
 'my_mnist_model/0002/variables/variables.data-00000-of-00001',
 'my_mnist_model/0002/variables/variables.index']

**경고**: 텐서플로 서빙이 새 모델을 로드하기까지 잠시 기다려야 할 수 있다.

In [18]:
server_url = 'http://172.17.0.3:8501/v1/models/my_mnist_model:predict'
response = post(server_url, request_json)
response.raise_for_status()
response = response.json()

In [19]:
response.keys()

dict_keys(['predictions'])

In [20]:
y_proba = array(response['predictions'])
y_proba.round(2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.99, 0.  , 0.01],
       [0.04, 0.  , 0.8 , 0.02, 0.  , 0.06, 0.06, 0.  , 0.02, 0.  ],
       [0.  , 0.93, 0.02, 0.01, 0.  , 0.  , 0.  , 0.01, 0.01, 0.  ]])

# 모바일 또는 임베디드 디바이스에 모델 배포하기

In [21]:
from tensorflow import lite

converter = lite.TFLiteConverter.from_saved_model(str(model_path))
tflite_model = converter.convert()
with open('my_converted_savedmodel.tflite', 'wb') as f:
    f.write(tflite_model)

W0000 00:00:1756179026.407785      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1756179026.407837      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-26 03:30:26.411063: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: my_mnist_model/0002
2025-08-26 03:30:26.414109: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-26 03:30:26.414121: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: my_mnist_model/0002
I0000 00:00:1756179026.418217      35 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-08-26 03:30:26.418814: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-26 03:30:26.442368: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: my_mnist_model/0002
2025-08-26 03:30:26.447803: I tensorflow/cc/saved_model/loader.cc:471] Sa

In [22]:
# 케라스 모델을 변환한다.
converter = lite.TFLiteConverter.from_keras_model(model)

In [23]:
converter.optimizations = [lite.Optimize.DEFAULT]

In [24]:
tflite_model = converter.convert()
with open('my_converted_keras_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpp3xus6sh/assets


INFO:tensorflow:Assets written to: /tmp/tmpp3xus6sh/assets


Saved artifact at '/tmp/tmpp3xus6sh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.uint8, name='keras_tensor_5')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  124754704634832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754704632336: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754709358800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754704633104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754704632912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  124754709359376: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1756179026.722468      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1756179026.722510      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-08-26 03:30:26.722660: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpp3xus6sh
2025-08-26 03:30:26.723063: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-08-26 03:30:26.723069: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpp3xus6sh
2025-08-26 03:30:26.725578: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-08-26 03:30:26.740786: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpp3xus6sh
2025-08-26 03:30:26.745809: I tensorflow/cc/saved_model/loader.cc:471] SavedModel load for tags { serve }; Status: success: OK. Took 23151 microseconds.


# GPU를 사용하여 계산 속도 향상하기

텐서플로우가 GPU를 볼 수 있는지 확인한다:

In [25]:
physical_gpus = list_physical_devices('GPU')
physical_gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

텐서플로 스크립트에서 GPU \#0과 \#1(PCI 순서 기준)만 사용하려면 스크립트를 시작하기 전에 환경 변수 `CUDA_DEVICE_ORDER=PCI_BUS_ID`와 `CUDA_VISIBLE_DEVICES=0,1`을 설정한다. 또는 스크립트 자체에서 텐서플로를 사용하기 전에 지정한다.

## GPU RAM 관리

RAM 용량을 GPU당 2GB로 제한하려면:

In [26]:
# for gpu in physical_gpus:
#     config.set_logical_device_configuration(gpu, [config.LogicalDeviceConfiguration(2048)])

점진적으로 텐서플로가 메모리를 점유하도록 하려면(프로세스가 종료될 때만 메모리를 해제한다):

In [27]:
from tensorflow.config.experimental import set_memory_growth

# for gpu in physical_gpus:
#     set_memory_growth(gpu, True)

이와 동일하게, 텐서플로를 사용하기 전에 `TF_FORCE_GPU_ALLOW_GROWTH` 환경 변수를 `true`로 설정할 수 있다.

물리적 GPU를 두 개의 논리적 GPU로 분할한다:

In [28]:
from tensorflow.config import LogicalDeviceConfiguration

# set_logical_device_configuration(physical_gpus[0], [LogicalDeviceConfiguration(2048), LogicalDeviceConfiguration(2048)])

In [29]:
from tensorflow.config import list_logical_devices

logical_gpus = list_logical_devices('GPU')
logical_gpus

[LogicalDevice(name='/device:GPU:0', device_type='GPU')]

## 디바이스에 연산 및 변수 배치하기

모든 변수 및 연산 배치를 기록하려면(이 작업은 텐서플로를 임포팅한 직후에 실행해야 한다):

In [30]:
from tensorflow import get_logger
from tensorflow.debugging import set_log_device_placement

get_logger().setLevel("DEBUG")  # 디폴트 로그 수준은 INFO다.
set_log_device_placement(True)

In [31]:
from tensorflow import Variable

a = Variable([1., 2., 3.])  # float32 변수는 GPU로 이동한다.
a.device

'/job:localhost/replica:0/task:0/device:GPU:0'

In [32]:
b = Variable([1, 2, 3])  # int32 변수는 CPU로 이동한다.
b.device

'/job:localhost/replica:0/task:0/device:CPU:0'

`tf.device()` 컨텍스트를 사용하여 원하는 장치에 변수 및 연산을 수동으로 배치할 수 있다:

In [33]:
from tensorflow import device

with device('/CPU:0'):
    c = Variable([1., 2., 3.])
c.device

'/job:localhost/replica:0/task:0/device:CPU:0'

존재하지 않거나 커널이 없는 장치를 지정하면 텐서플로는 자동으로 기본 장치를 사용한다:

In [34]:
with device('/GPU:1234'):
    d = Variable([1., 2., 3.])
d.device

'/job:localhost/replica:0/task:0/device:GPU:0'

텐서플로에서 존재하지 않는 장치를 사용하려고 할 때 기본 장치로 되돌아가지 않고 예외를 발생시키려면:

In [35]:
from tensorflow.config import set_soft_device_placement
from tensorflow.errors import InvalidArgumentError

set_soft_device_placement(False)
try:
    with device('/GPU:1000'):
        d = Variable([1., 2., 3.])
except InvalidArgumentError as ex:
    print(ex)
set_soft_device_placement(True)  # 소프트 배치로 돌아가기

Could not satisfy device specification '/job:localhost/replica:0/task:0/device:GPU:1000'. enable_soft_placement=0. Supported device types [GPU, CPU]. All available devices [/job:localhost/replica:0/task:0/device:GPU:0, /job:localhost/replica:0/task:0/device:CPU:0].


## 여러 디바이스에서 병렬 실행

inter-op 또는 intra-op 스레드 수를 설정하려는 경우(CPU 포화를 방지하거나 완벽하게 재현 가능한 테스트 케이스를 실행하기 위해 텐서플로를 단일 스레드로 만들고자 하는 경우 유용하다):

In [36]:
from tensorflow.config.threading import set_inter_op_parallelism_threads

# set_inter_op_parallelism_threads(10)
# set_intra_op_parallelism_threads(10)

# 여러 디바이스에서 모델 훈련하기

## 분산 전략 API를 사용한 대규모 훈련

In [37]:
from tensorflow.keras.layers import Reshape, Conv2D, MaxPool2D, Dropout


# 케라스를 사용하여 MNIST용 CNN 모델을 생성한다.
def create_model():
    return Sequential([
        Input((28, 28), dtype=uint8),
        Reshape([28, 28, 1]),
        Rescaling(1 / 255),
        Conv2D(64, 7, padding='same', activation='relu'),
        MaxPool2D(2),
        Conv2D(128, 3, padding='same', activation='relu'),
        Conv2D(128, 3, padding='same', activation='relu'),
        MaxPool2D(2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(.5),
        Dense(10, activation='softmax')
    ])

In [38]:
from tensorflow import distribute

strategy = distribute.MirroredStrategy()
with strategy.scope():
    model = create_model()  # 일반적인 케라스 모델을 생성한다.
    model.compile('SGD', 'sparse_categorical_crossentropy', metrics=['accuracy'])  # 모델을 컴파일한다.
batch_size = 100  # 복제본 수로 나누어지는 것이 바람직하다.
model.fit(X_train, y_train, batch_size, 1, validation_data=(X_valid, y_valid))  # epochs=10

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


550/550 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - accuracy: 0.5760 - loss: 1.3121 - val_accuracy: 0.9094 - val_loss: 0.3285


2025-08-26 03:30:34.584449: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node MultiDeviceIteratorGetNextFromShard}}]]
	 [[RemoteCall]]


In [39]:
model.distribute_strategy

In [40]:
model.predict(X_new).round(2)  # 배치가 모든 복제본에 분할된다.

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step


array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  , 0.  , 0.  ],
       [0.  , 0.  , 0.99, 0.  , 0.  , 0.  , 0.01, 0.  , 0.  , 0.  ],
       [0.  , 0.96, 0.01, 0.  , 0.  , 0.  , 0.01, 0.  , 0.01, 0.  ]],
      dtype=float32)

In [41]:
from tensorflow.keras.models import load_model

# 모델을 저장해도 분산 전략이 보존되지 않는다.
model.save('my_mirrored_model.keras')
model = load_model('my_mirrored_model.keras')
model.distribute_strategy

In [42]:
with strategy.scope():
    model = load_model('my_mirrored_model.keras')

In [43]:
model.distribute_strategy

사용할 GPU 리스트를 지정하려는 경우:

In [44]:
strategy = distribute.MirroredStrategy(['/GPU:0', '/GPU:1'])

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


기본 all-reduce 알고리즘을 변경하려는 경우:

In [45]:
strategy = distribute.MirroredStrategy(cross_device_ops=distribute.HierarchicalCopyAllReduce())

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


`CentralStorageStrategy`을 사용하려는 경우:

In [46]:
strategy = distribute.experimental.CentralStorageStrategy()

INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:GPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:GPU:0'


INFO:tensorflow:ParameterServerStrategy (CentralStorageStrategy if you are using a single machine) with compute_devices = ['/job:localhost/replica:0/task:0/device:GPU:0'], variable_device = '/job:localhost/replica:0/task:0/device:GPU:0'


In [47]:
distribute.cluster_resolver.TPUClusterResolver

tensorflow.python.distribute.cluster_resolver.tpu.tpu_cluster_resolver.TPUClusterResolver

In [48]:
from sys import modules
from os import environ
from tensorflow.config import experimental_connect_to_cluster
from tensorflow.tpu.experimental import initialize_tpu_system

# Google Colab에서 TPU로 훈련하기:
if 'google.colab' in modules and 'COLAB_TPU_ADDR' in environ:
    tpu_address = 'grpc://' + environ['COLAB_TPU_ADDR']
else:
    tpu_address = ''
# resolver = distribute.cluster_resolver.TPUClusterResolver(tpu_address)
# experimental_connect_to_cluster(resolver)
# initialize_tpu_system(resolver)
# strategy = distribute.TPUStrategy(resolver)

## 텐서플로 클러스터에서 모델 훈련하기

텐서플로 클러스터는 일반적으로 서로 다른 컴퓨터에서 병렬로 실행되며 신경망 훈련이나 실행과 같은 작업을 완료하기 위해 서로 대화하는 텐서플로 프로세스의 그룹이다. 클러스터의 각 TF 프로세스를 "태스크"(또는 "TF 서버")라고 한다. IP 주소, 포트, 타입(역할 또는 작업이라고도 함)이 있다. 타입은 `"worker"`, `"chief"`, `"ps"`(파라미터 서버) 또는 `"evaluator"`가 될 수 있다:
* **워커**는 일반적으로 하나 이상의 GPU가 있는 컴퓨터에서 계산을 수행한다.
* **치프**는 계산도 수행하지만, 텐서보드 로그 작성이나 체크포인트 저장과 같은 추가 작업도 처리한다. 클러스터에는 하나의 치프가 있다. 정의되지 않은 경우 워커 #0이 치프가 된다.
* **파라미터 서버**(ps)는 변수 값만 추적하며, 일반적으로 CPU 전용 머신에 있다.
* **이밸류에이터**는 당연히 평가를 담당한다. 일반적으로 클러스터에는 하나의 이밸류에이터가 있다.

같은 유형의 태스크 집합을 흔히 "작업(job)"이라고 한다. 예를 들어 "워커" 작업은 모든 워커 태스크의 집합이다.

텐서플로 클러스터를 시작하려면 먼저 클러스터를 정의해야 한다. 즉, 모든 작업(IP 주소, TCP 포트 및 타입)을 지정해야 한다. 예를 들어, 다음 클러스터 사양은 3개의 태스크(워커 2개, 파라미터 서버 1개)가 있는 클러스터를 정의한다. 작업당 하나의 키가 있는 사전이며, 값은 태스크 주소 목록이다:

In [49]:
cluster_spec = {
    'worker': ['machine-a.example.com:2222', 'machine-b.example.com:2222'], 'ps': ['machine-a.example.com:2221']
    # [/job:worker/task:0, /job:worker/task:1], [/job:ps/task:0]
}

In [50]:
environ['TF_CONFIG'] = dumps({'cluster': cluster_spec, 'task': {'type': 'worker', 'index': 0}})

일부 플랫폼(예: 구글 버텍스 AI)에서는 이 환경 변수를 자동으로 설정한다.

텐서플로의 `TFConfigClusterResolver` 클래스는 이 환경 변수에서 클러스터 구성을 읽는다:

In [51]:
resolver = distribute.cluster_resolver.TFConfigClusterResolver()
resolver.cluster_spec()

ClusterSpec({'ps': ['machine-a.example.com:2221'], 'worker': ['machine-a.example.com:2222', 'machine-b.example.com:2222']})

In [52]:
resolver.task_type

'worker'

In [53]:
resolver.task_id

0

이제 로컬 컴퓨터에서 두 개의 워커 태스크를 가진 간단한 클러스터를 실행한다. `MultiWorkerMirroredStrategy`을 사용하여 두 태스크로 모델을 훈련한다.

첫 번째 단계는 훈련 코드를 작성하는 것이다. 이 코드를 사용해 두 워커에서 각각 고유한 프로세스로 실행하므로 별도의 파이썬 파일인 `my_mnist_multiworker_task.py`에 작성한다. 코드는 비교적 간단하지만 주의해야 할 몇 가지 중요한 사항이 있다:
* 텐서플로로 다른 작업을 수행하기 전에 `MultiWorkerMirroredStrategy`을 생성한다.
* 워커 중 하나만 텐서보드 로깅을 처리한다. 앞서 언급했듯이 이 작업자를 *치프*라고 한다. 명시적으로 정의되지 않은 경우 워커 #0이 치프다.

In [54]:
%%writefile my_mnist_multiworker_task.py
from tensorflow import distribute
from tf_keras.datasets.mnist import load_data
from tf_keras import Sequential, Input
from tensorflow import uint8
from tf_keras.layers import Reshape, Rescaling, Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tempfile import mkdtemp
from tensorflow.io.gfile import rmtree

strategy = distribute.MultiWorkerMirroredStrategy()  # 시작 부분에!
resolver = distribute.cluster_resolver.TFConfigClusterResolver()
print(f'Starting task {resolver.task_type} #{resolver.task_id}')
# MNIST 데이터셋 로드 및 분할
mnist = load_data()
(X_train_full, y_train_full), (X_test, y_test) = mnist
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
with strategy.scope():
    model = Sequential([
        Input((28, 28), dtype=uint8),
        Reshape((28, 28, 1)),
        Rescaling(1 / 255),
        Conv2D(64, 7, padding='same', activation='relu'),
        MaxPool2D(2),
        Conv2D(128, 3, padding='same', activation='relu'),
        Conv2D(128, 3, padding='same', activation='relu'),
        MaxPool2D(2),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(.5),
        Dense(10, activation='softmax')
    ])
    model.compile('SGD', 'sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))  # epochs=10
if resolver.task_id == 0:  # 치프는 모델을 올바른 위치에 저장한다.
    model.export('my_mnist_multiworker_model')
else:
    tmpdir = mkdtemp()  # 다른 워커는 임시 디렉터리에 저장한다.
    model.export(tmpdir)
    rmtree(tmpdir)  # 마지막에 이 디렉터리를 삭제할 수 있다!

Overwriting my_mnist_multiworker_task.py


실제 애플리케이션에서는 일반적으로 머신당 하나의 워커가 있지만, 이 예제에서는 동일한 머신에서 두 워커를 모두 실행하고 있으므로 두 워커 모두 사용 가능한 모든 GPU RAM(이 머신에 GPU가 있는 경우)을 사용하려고 시도하므로 메모리 부족(OOM) 오류가 발생할 수 있다. 이를 방지하기 위해 `CUDA_VISIBLE_DEVICES` 환경 변수를 사용하여 각 워커에 다른 GPU를 할당한다. 또는 `CUDA_VISIBLE_DEVICES`를 빈 문자열로 설정하여 간단히 GPU 지원을 비활성화한다.

In [55]:
%%bash --bg
export CUDA_VISIBLE_DEVICES=''
export TF_CONFIG='{"cluster": {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]}, "task": {"type": "worker", "index": 0}}'
python my_mnist_multiworker_task.py &> my_worker_0.log

In [56]:
%%bash --bg
export CUDA_VISIBLE_DEVICES=''
export TF_CONFIG='{"cluster": {"worker": ["127.0.0.1:9901", "127.0.0.1:9902"]}, "task": {"type": "worker", "index": 1}}'
python my_mnist_multiworker_task.py &> my_worker_1.log

**참고**: `AutoShardPolicy`에 대한 경고가 표시되면 무시해도 무방하다. 자세한 내용은 [TF 이슈 #42146](https://github.com/tensorflow/tensorflow/issues/42146)을 참고한다.

끝났다! 이제 텐서플로 클러스터가 실행 중이지만 별도의 프로세스에서 실행 중이므로 이 노트북에서는 볼 수 없다(하지만 진행 상황은 `my_worker_*.log`에서 확인할 수 있다).

In [67]:
# strategy = distribute.MultiWorkerMirroredStrategy(communication_options=distribute.experimental.CommunicationOptions(
#     implementation=distribute.experimental.CommunicationImplementation.NCCL
# ))